In [16]:
import sys
sys.path.append('./src')
import dataflow
import model as models
import utils

import lightning as L
from omegaconf import OmegaConf

cfg_path = 'config.yaml'
cfg = OmegaConf.load(cfg_path)

dm = dataflow.TweetDataModule(cfg)
dm.prepare_data()

# splits/transforms
dm.setup(stage="fit")

cfg.data_processing.label_decode_map = dm.label_decode_map
cfg.data_processing.tokenizer.kwargs.vocab_size = dm.tokenizer.vocab_size
cfg.hyperparameters.backbone.kwargs.input_size = dm.tokenizer.vocab_size

sample = next(iter(dm.train_dataloader()))


Train/Val/Test Splits: 8784/2928/2928


Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [19]:
model = models.SentimentClassifier.load_from_checkpoint('wandb-sentiment-analysis/0vfrmtju/checkpoints/epoch=4-step=2745.ckpt', tokenizer=dm.tokenizer, hyperparams=cfg.hyperparameters)


2023-05-20 23:11:53,781 model INFO: Frozen the first 123 out of 246 encoder weights


In [23]:
logits = model.get_logits(sample[0])
logits

tensor([[ 0.5837,  0.5044, -1.5180],
        [ 2.0489, -0.8427, -2.4948],
        [ 2.5640, -1.6338, -2.7393],
        [ 0.3040,  0.7500, -1.3418],
        [ 0.1069,  0.0412, -0.2656],
        [ 2.0290, -0.7391, -2.4948],
        [ 1.4048, -0.0654, -2.2017],
        [ 1.9650, -0.6771, -2.4681],
        [ 0.4626, -0.3696, -0.2962],
        [ 3.1427, -1.2991, -3.8980],
        [ 5.0072, -3.1753, -5.6953],
        [ 0.8798, -2.1245,  0.2674],
        [ 0.1739,  1.1039, -1.5528],
        [ 2.8586, -1.5739, -3.1581],
        [-0.7312,  1.0878, -0.4434],
        [ 1.4208, -0.5154, -1.7409]], grad_fn=<AddmmBackward0>)

In [25]:
sample[1]

tensor([1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0])

In [24]:
import torch
torch.argmax(logits, dim=1).flatten().tolist()

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]